# Parte 1: Dados - Users_Followers

In [25]:
import pandas as pd
import numpy as np
import re

def fs_to_dataframe(fs):
    return pd.DataFrame({'User': re.split('\W+', str(fs))})


In [26]:
# load dataset

gh_users_followers = pd.read_csv('../data/users__followers.csv')
gh_users_followers.head()

,User,All_Followers,nF
0,tarruda,"Sannis,danielmahon,csjaba,FergusRedican,Victor...",570
1,mairatma,"brunocoelho,henvic,eduardolundgren,aperrelli,a...",363
2,joselitojunior1,"renatooliveira,jeffesonmaia,jotaefe,duartefq,J...",350
3,marcelcaraciolo,"thiagoarrais,brunojm,henriquebastos,macndesign...",330
4,luanfonceca,"brunohenrique,luizvarela,gladson,lucasbibiano,...",301


In [27]:
# normalizing

gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].fillna('')
gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].apply(fs_to_dataframe) # apenas para simplificar operacoes 

gh_users_allFollowers.head()

,User,All_Followers,nF
0,tarruda,User 0 1 ...,570
1,mairatma,User 0 1 ...,363
2,joselitojunior1,User 0 1 ...,350
3,marcelcaraciolo,User 0 1 ...,330
4,luanfonceca,User 0 1 ...,301


In [28]:
# Dataframe incluindo colunas contendo apenas seguidores recifenses

gh_users_Followers = gh_users_allFollowers.copy()

gh_users_Followers['Followers'] = gh_users_Followers['All_Followers'].apply(pd.DataFrame({'User':gh_users_Followers['User']}).merge)

gh_users_Followers['nFs'] = gh_users_Followers['Followers'].apply(len)

gh_users_Followers.head()

,User,All_Followers,nF,Followers,nFs
0,tarruda,User 0 1 ...,570,Empty DataFrame Columns: [User] Index: [],0
1,mairatma,User 0 1 ...,363,User 0 henvic 1 aperrelli,2
2,joselitojunior1,User 0 1 ...,350,User 0 renatooliveira 1 karl...,5
3,marcelcaraciolo,User 0 1 ...,330,User 0 gileno 1 fjs...,8
4,luanfonceca,User 0 1 ...,301,User 0 filipeximenes 1 renatoo...,2


In [29]:
# foco na visualização dos Followers recifenses 
# (mantendo no DataFrame todos os seguidores(recifenses e não recifenses))

gh_users_Followers = gh_users_Followers.loc[:, ['User', 'Followers', 'nFs', 'All_Followers', 'nF']]
gh_users_Followers.sort_values('nFs', axis=0, ascending=False, inplace=True)

gh_users_Followers.head()

,User,Followers,nFs,All_Followers,nF
29,guiocavalcanti,User 0 deividazevedo2 1 ...,24,User 0 1 ...,58
26,lailsonbm,User 0 luanfonceca 1 ...,23,User 0 1 ...,73
8,filipeximenes,User 0 luanfonceca ...,23,User 0 ...,177
23,guilhermefarias,User 0 filipeximenes 1 ...,19,User 0 1 ...,80
12,fernandocastor,User 0 joselitojunior1 1 ...,19,User 0 1 ...,129


In [30]:
gh_users_Followers['Followers'] = gh_users_Followers['Followers'].apply(lambda x: x.apply(','.join))
gh_users_Followers['All_Followers'] = gh_users_Followers['All_Followers'].apply(lambda x: x.apply(','.join))

gh_users_Followers.head()

,User,Followers,nFs,All_Followers,nF
29,guiocavalcanti,"deividazevedo2,filipeximenes,renatooliveira,he...",24,",User,0,charlenopires,1,brunnogomes,2,adrianom...",58
26,lailsonbm,"luanfonceca,gileno,renatooliveira,fjsj,guiocav...",23,",User,0,kraudio,1,lucasmncastro,2,chillicoder,...",73
8,filipeximenes,"luanfonceca,gileno,renatooliveira,luiztiago,fj...",23,",User,0,luisgabriel,1,renatooliveira,2,marciob...",177
23,guilhermefarias,"filipeximenes,luiztiago,henvic,talitaoliveira,...",19,",User,0,ivanAndrade,1,dodiegofly,2,iurimattos,...",80
12,fernandocastor,"joselitojunior1,filipeximenes,renatooliveira,f...",19,",User,0,guiocavalcanti,1,adrianomelo,2,tacsio,...",129


In [31]:
# todos os usuarios recifenses (nem todos serao considerados como nos do graph - ver Parte 2) 
nodes_preview = gh_users_Followers.loc[:,['User','nFs', 'nF']]
nodes_preview.columns = ['Id', 'In-Degree', 'All-In-Degree']

nodes_preview.head()

,Id,In-Degree,All-In-Degree
29,guiocavalcanti,24,58
26,lailsonbm,23,73
8,filipeximenes,23,177
23,guilhermefarias,19,80
12,fernandocastor,19,129


In [7]:
# criacao das arestas

def create_edges_df(batches):
    return pd.concat(batches,
        ignore_index=True)

pairs = zip(gh_users_Followers.User, gh_users_Followers.Followers)

edges = [pd.DataFrame([{'Source':str(f), 'Target':str(u)} for f in fs['User']]) for (u,fs) in pairs]

edges_df = create_edges_df(edges)
edges_df.head()

,Source,Target
0,henriquemenezes,tarruda
1,paulolieuthier,tarruda
2,meiralins,tarruda
3,dapine,tarruda
4,spider8,tarruda


In [10]:
# gh_users_Followers.to_csv('gh_users_Followers.csv', index=False)
# edges_df.to_csv('edges.csv', index=False)


# Parte 2: Análise dos Dados - Users_Followers

In [11]:
# numero de usuarios recifenses 
gh_users_Followers['User'].count()

1992

In [69]:
# numero de usuarios recifenses com seguidores recifenses 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] != 0]]
len(users_0_fs)

692

In [13]:
# numero de usuarios recifenses sem seguidores recifenses
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] == 0]]
len(users_0_fs)

1300

In [70]:
# numero de usuarios recifenses com seguidores (recifenses ou nao)
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nF'] != 0]]
len(users_0_fs)

1103

In [71]:
# numero de usuarios recifenses sem seguidores (recifenses ou nao)
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nF'] == 0]]
len(users_0_fs)

889

**- Obs.: número de nós considerados**

Note que como estes nós pertencem a uma aresta, eles representam os usuários que tem seguidores e/ou seguem alguém.

In [14]:
# (usuarios recifenses, excluidos os que não tem seguidores se e somente se não seguem ninguem)
len(pd.unique(edges_df[['Source','Target']].values.ravel()))

828

In [15]:
# numero de arestas (somatorio do total de seguidores por usuario)
gh_users_Followers.nFs.sum()

2320

In [16]:
# revendo o numero de arestas
edges = pd.read_csv('edges.csv')
edges.count()

Source    2320
Target    2320
dtype: int64

In [17]:
# distribuicao do Grau
gh_users_Followers.nFs.value_counts()

0     1300
1      310
2      142
3       67
4       42
5       27
7       19
6       18
8       14
9       14
11       6
14       5
12       3
23       3
20       2
13       2
18       2
10       2
15       2
24       2
25       2
29       1
37       1
35       1
16       1
28       1
19       1
40       1
48       1
Name: nFs, dtype: int64

In [ ]:
# distribuicao do grau - usuarios por ocorrencias da quantidade de seguidores 
groupby_nfs = gh_users_Followers.groupby('nFs')
groupby_nfs['User'].apply( ','.join).sort()